In [ ]:
!pip install s3fs
!pip install sagemaker

In [ ]:
BUCKET='ml-lab-pyspark'
!export AWS_PROFILE=ml-lab

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime as dt

In [ ]:
deals=pd.read_csv('../data/deals_replay.csv',
    header=None,
    names=["timestamp","deal_type","order_type","stop","limit","price","stop_price","limit_price","loss_or_profit"],
    dtype={
        "timestamp":str
    }
)

## All orders sorted by timestamp

In [ ]:
deals.sort_values(by="timestamp")

### All `buy` type orders where the stop/limit was 10

In [ ]:
qualified_deals=deals.query('order_type=="buy" & stop==10')
qualified_deals

In [ ]:
qualified_deals.describe()

Among above orders, below have made a profit. We want to predict these orders:

In [ ]:
len(qualified_deals.query('deal_type=="limit"'))

In [ ]:
price=pd.read_csv('../data/price.csv',
    header=None,
    names=["datetime","timestamp","MID_OPEN","BID","OFFER","CHANGE","CHANGE_PCT","HIGH","LOW"],
    dtype={
        "timestamp":str,
        "datetime":str
    }
)

In [ ]:
price.sort_values(by="timestamp")

In [ ]:
deals_with_price=pd.merge(qualified_deals, price,how="left",on="timestamp",sort=True)
deals_with_price['MID']=(deals_with_price['BID']+deals_with_price['OFFER'])/2
deals_with_price

In [ ]:
_format='%y-%m-%d %H:%M:%S'

def datetime_to_int(x):
    return int(dt.strptime(x,_format).timestamp())

datetime_to_int('20-03-13 11:47:24')

In [ ]:
from datetime import datetime as dt
deals_with_price['datetime_int']=deals_with_price['datetime'].apply(datetime_to_int)
deals_with_price

In [ ]:
mean_price_each_second=deals_with_price[['datetime','BID','OFFER']].groupby('datetime').mean().reset_index()
mean_price_each_second['datetime_int']=mean_price_each_second['datetime'].apply(datetime_to_int)
mean_price_each_second['MID']=(mean_price_each_second['BID']+mean_price_each_second['OFFER'])/2
mean_price_each_second

In [ ]:
df=pd.DataFrame()
historical_data_points=600
for i,row in deals_with_price.iterrows():
    values=np.full(historical_data_points,np.nan)
    _timestamp=row['datetime_int']
    prices=mean_price_each_second[['datetime_int','MID']].query('datetime_int<%s' % _timestamp).tail(historical_data_points)['MID']
    values[0:len(prices)]=prices.values
    value_series=pd.Series(values)
    df.insert(loc=i,column=str(row['timestamp']),value=values)
df.transpose()

In [ ]:
all_data_absolute=deals_with_price[['timestamp','BID','OFFER','MID_OPEN','HIGH','LOW']].merge(df.transpose(),left_on='timestamp',right_index=True)
all_data_absolute

In [ ]:
all_data_absolute.drop('timestamp',axis=1,inplace=True)

In [253]:
all_data_absolute.fillna(method='backfill',inplace=True)
all_data_absolute.fillna(method='pad',inplace=True)
all_data_absolute

,BID,OFFER,MID_OPEN,HIGH,LOW,0,1,2,3,4,...,590,591,592,593,594,595,596,597,598,599
0,1.25176,1.25193,1.25699,1.26256,1.25052,1.251845,1.251895,1.251923,1.251871,1.251793,...,1.251036,1.251046,1.251061,1.251081,1.251231,1.251271,1.251271,1.251262,1.251262,1.251355
1,1.25177,1.25194,1.25699,1.26256,1.25052,1.251845,1.251895,1.251923,1.251871,1.251793,...,1.251036,1.251046,1.251061,1.251081,1.251231,1.251271,1.251271,1.251262,1.251262,1.251355
2,1.25180,1.25200,1.25699,1.26256,1.25052,1.251845,1.251895,1.251923,1.251871,1.251793,...,1.251036,1.251046,1.251061,1.251081,1.251231,1.251271,1.251271,1.251262,1.251262,1.251355
3,1.25184,1.25202,1.25699,1.26256,1.25052,1.251845,1.251895,1.251923,1.251871,1.251793,...,1.251036,1.251046,1.251061,1.251081,1.251231,1.251271,1.251271,1.251262,1.251262,1.251355
4,1.25182,1.25202,1.25699,1.26256,1.25052,1.251845,1.251895,1.251923,1.251871,1.251793,...,1.251036,1.251046,1.251061,1.251081,1.251231,1.251271,1.251271,1.251262,1.251262,1.251355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7166,1.24964,1.24981,1.25699,1.26256,1.24884,1.252425,1.252413,1.252412,1.252416,1.252200,...,1.249884,1.249825,1.249830,1.249850,1.249835,1.249805,1.249739,1.249710,1.249701,1.249690
7167,1.24963,1.24980,1.25699,1.26256,1.24884,1.252425,1.252413,1.252412,1.252416,1.252200,...,1.249884,1.249825,1.249830,1.249850,1.249835,1.249805,1.249739,1.249710,1.249701,1.249690
7168,1.24964,1.24981,1.25699,1.26256,1.24884,1.252425,1.252413,1.252412,1.252416,1.252200,...,1.249884,1.249825,1.249830,1.249850,1.249835,1.249805,1.249739,1.249710,1.249701,1.249690
7169,1.24964,1.24981,1.25699,1.26256,1.24884,1.252425,1.252413,1.252412,1.252416,1.252200,...,1.249884,1.249825,1.249830,1.249850,1.249835,1.249805,1.249739,1.249710,1.249701,1.249690


In [254]:
mid=(all_data_absolute['BID']+all_data_absolute['OFFER'])/2
mid

0       1.251845
1       1.251855
2       1.251900
3       1.251930
4       1.251920
          ...   
7166    1.249725
7167    1.249715
7168    1.249725
7169    1.249725
7170    1.249725
Length: 7171, dtype: float64

In [255]:
all_data=all_data_absolute.subtract(mid,axis='index')
all_data

,BID,OFFER,MID_OPEN,HIGH,LOW,0,1,2,3,4,...,590,591,592,593,594,595,596,597,598,599
0,-0.000085,0.000085,0.005145,0.010715,-0.001325,0.000000,0.000050,0.000078,0.000026,-0.000052,...,-0.000809,-0.000799,-0.000784,-0.000764,-0.000614,-0.000574,-0.000574,-0.000582,-0.000583,-0.000490
1,-0.000085,0.000085,0.005135,0.010705,-0.001335,-0.000010,0.000040,0.000068,0.000016,-0.000062,...,-0.000819,-0.000809,-0.000794,-0.000774,-0.000624,-0.000584,-0.000584,-0.000592,-0.000593,-0.000500
2,-0.000100,0.000100,0.005090,0.010660,-0.001380,-0.000055,-0.000005,0.000023,-0.000029,-0.000107,...,-0.000864,-0.000854,-0.000839,-0.000819,-0.000669,-0.000629,-0.000629,-0.000638,-0.000638,-0.000545
3,-0.000090,0.000090,0.005060,0.010630,-0.001410,-0.000085,-0.000035,-0.000008,-0.000059,-0.000137,...,-0.000894,-0.000884,-0.000869,-0.000849,-0.000699,-0.000659,-0.000659,-0.000668,-0.000668,-0.000575
4,-0.000100,0.000100,0.005070,0.010640,-0.001400,-0.000075,-0.000025,0.000002,-0.000049,-0.000127,...,-0.000884,-0.000874,-0.000859,-0.000839,-0.000689,-0.000649,-0.000649,-0.000658,-0.000658,-0.000565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7166,-0.000085,0.000085,0.007265,0.012835,-0.000885,0.002700,0.002688,0.002687,0.002691,0.002475,...,0.000159,0.000100,0.000105,0.000125,0.000110,0.000080,0.000014,-0.000015,-0.000024,-0.000035
7167,-0.000085,0.000085,0.007275,0.012845,-0.000875,0.002710,0.002698,0.002697,0.002701,0.002485,...,0.000169,0.000110,0.000115,0.000135,0.000120,0.000090,0.000024,-0.000005,-0.000014,-0.000025
7168,-0.000085,0.000085,0.007265,0.012835,-0.000885,0.002700,0.002688,0.002687,0.002691,0.002475,...,0.000159,0.000100,0.000105,0.000125,0.000110,0.000080,0.000014,-0.000015,-0.000024,-0.000035
7169,-0.000085,0.000085,0.007265,0.012835,-0.000885,0.002700,0.002688,0.002687,0.002691,0.002475,...,0.000159,0.000100,0.000105,0.000125,0.000110,0.000080,0.000014,-0.000015,-0.000024,-0.000035


In [256]:
labels=deals_with_price['deal_type'].apply(lambda x:1 if x=='limit' else 0)
labels

0       0
1       0
2       0
3       0
4       0
       ..
7166    1
7167    1
7168    1
7169    1
7170    1
Name: deal_type, Length: 7171, dtype: int64

In [257]:
count=len(all_data)

import math
training=math.floor(0.7*count)
test=math.floor(0.15*count)
validation=math.floor(0.15*count)
evaluation=math.floor(0.2*count)

In [258]:
training_data_set=all_data[0:training]
training_labels=labels[0:training]
len(training_data_set)

5019

In [259]:
test_data_set=all_data[(training+1):(training+test)]
test_labels=labels[(training+1):(training+test)]
len(test_data_set)

1074

In [260]:
validation_data_set=all_data[(training+test+1):]
validation_labels=labels[(training+test+1):]
len(validation_data_set)

1076

In [261]:
training_data_set.to_csv('../data/training_data.csv',header=None,index=False)
test_data_set.to_csv('../data/test_data.csv',header=None,index=False)
validation_data_set.to_csv('../data/validation_data.csv',header=None,index=False)

In [262]:
training_data_np=training_data_set.to_numpy().astype('float32')
training_labels_np=training_labels.to_numpy().astype('float32')

test_data_np=test_data_set.to_numpy().astype('float32')
test_labels_np=test_labels.to_numpy().astype('float32')

validation_data_np=validation_data_set.to_numpy().astype('float32')
validation_labels_np=validation_labels.to_numpy().astype('float32')

In [263]:
import io
import sagemaker.amazon.common as smac
import boto3
session=boto3.Session(profile_name='ml-lab')

In [264]:
buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, training_data_np, training_labels_np)
buf.seek(0)
session.resource('s3').Bucket(BUCKET).Object('training_data.io').upload_fileobj(buf)

In [265]:
buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, test_data_np, test_labels_np)
buf.seek(0)
session.resource('s3').Bucket(BUCKET).Object('test_data.io').upload_fileobj(buf)

In [266]:
buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, validation_data_np, validation_labels_np)
buf.seek(0)
session.resource('s3').Bucket(BUCKET).Object('validation_data.io').upload_fileobj(buf)

In [267]:
evaluation_data=all_data.copy()
evaluation_data['profit']=labels
evaluation_data_set=evaluation_data[(training+test+1):]

In [268]:
evaluation_data_set.to_csv('../data/evaluation_data.csv',header=None,index=False)